In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from src.jetnet import JetNetFeatures
from torch.utils.data import DataLoader, Dataset
import os
import h5py
import torch
from tabulate import tabulate

def Bins(xmin, xmax, bins=100):
    dx = (xmax - xmin) / bins
    return np.arange(xmin, xmax, dx)

In [ ]:
class DataConfig:

    jet_type : str = 'tops'

    sets : dict = {'diff_ddim'   : ('ddim_200.h5', 'etaphipt_frac'),
                    'fm_midpoint' : ('fm_tops150_mp200nfe.h5', 'etaphipt'),
                   }

    labels : dict = {'diff_ddim'   : 0,
                     'fm_midpoint' : 1,
                     }

    config = DataConfig 


In [ ]:
# class JetNetDataSets(Dataset):

#     ''' Arguments:
#         - `dir_path` : path to data files
#         - `data_files` : dictionary of data files to load, default is `None`
#         - `data_class_labels` : dictionary of class labels for each data file, default is `None`
#         - `num_jets`: number of jets to load, default is `None`
#         - `num_constituents`: number of particle constituents in each jet, default is `150`
#         - `particle_features`: list of particle features to include in data, default is `['eta_rel', 'phi_rel', 'pt_rel']`
#         - `preprocess`: dictionary of preprocessing methods to apply to data, default is `None`
#         - `clip_neg_pt`: clip negative pt values to zero, default is `False`
    
#         Loads and formats data from data files in format `.npy` or `.hdf5`.\\
#         Adds mask and zero-padding if necessary.\\
#         Adds class label for each sample if `data_class_labels` is provided.\\
#         Input data in files should always have shape (`#jets`, `#particles`, `#features`) with feaures: `(eta_rel, phi_rel, pt_rel, ...)`
        
#     '''
    
#     def __init__(self, 
#                  dir_path: str=None, 
#                  data_files: dict=None,
#                  data_class_labels: dict=None,
#                  num_jets: int=None,
#                  num_constituents: int=150,
#                  particle_features: list=['eta_rel', 'phi_rel', 'pt_rel'],
#                  preprocess: dict=None,
#                  clip_neg_pt: bool=False):
        
#         self.path = dir_path
#         self.data_files = data_files
#         self.data_class_labels = data_class_labels
#         self.num_jets = num_jets
#         self.num_consts = num_constituents
#         self.particle_features = particle_features
#         self.preprocess = preprocess 
#         self.clip_neg_pt = clip_neg_pt
#         self.data_list = self.load_data_list()
        
#     def __len__(self):
#         return len(self.data_list)
    
#     def __getitem__(self, idx):

#         name, filename, key, jet_idx, label = self.data_list[idx]
#         file_data = self.load_file(filename, key)
#         data = torch.Tensor(file_data[jet_idx])

#         N, D = data.shape
            
#         #...fill with zero padding up to 'num_constituents'

#         if D < 3: raise ValueError('Data has wrong shape: {}'.format(data.shape))
#         elif N < self.num_consts: 
#             zero_rows = torch.zeros(self.num_consts - N, D)
#             data = torch.cat((data, zero_rows), dim=0) 

#         #...get particle features

#         pf = {}
#         pf['eta_rel'] = data[:, 0, None]
#         pf['phi_rel'] = data[:, 1, None]
#         pf['pt_rel'] = data[:, 2, None] 
#         pf['log_pt_rel'] = torch.log(pf['pt_rel'])
#         pf['R'] = torch.sqrt(pf['eta_rel']**2 + pf['phi_rel']**2)
#         pf['e_rel'] = pf['pt_rel'] * torch.cosh(pf['eta_rel'])
#         pf['log_e_rel'] = torch.log(pf['e_rel'])
#         features = [pf[f] for f in self.particle_features]

#         #...add mask to indicate which particles are padded

#         mask = (data[:, 0] + data[:, 1] + data[:, 2] != 0).int().unsqueeze(-1)
#         features += [mask]
#         data = torch.cat(features, dim=-1)    
#         data = self._pt_order(data)

#         #...clip negative pt values

#         if self.clip_neg_pt:
#             data_clip = torch.clone(data)    
#             data = torch.zeros_like(data)
#             data[data_clip[:, 2] >= 0.0] = data_clip[data_clip[:, 2] >= 0.0]
#             data = self._pt_order(data)

#         #...preprocess data

#         if self.preprocess:
#             # TODO implement preprocessing
#             pass

#         #...remove softest jet constituents if more than 'num_constituents'

#         data = data[:self.num_consts]
        
#         if label is None: 
#             return data
#         else:
#             labels = torch.tensor(label, dtype=torch.long)  # Ensure label is a tensor
#             return data, labels

#     def _pt_order(self, data):
#         ''' Order particles in decreasing pt '''
#         return data[torch.argsort(data[..., 2], descending=True)]

#     def log_transform(self, data, features=['pt_rel']):
#         ''' Apply log transform to selected features in jet '''
#         mask = data[:, -1, None].bool()
#         idx = {'eta_rel':0, 'phi_rel':1, 'pt_rel':2, 'R':3, 'e_rel':4}
#         for f in features:
#             data[:, idx[f]] = torch.log(data[:, idx[f]])
#         return data
        
#     def load_file(self, filename, key):
#         filename = os.path.join(self.path, filename)
#         if filename.endswith('.npy'):
#             return np.load(filename)
#         elif filename.endswith('.hdf5'):
#             with h5py.File(filename, 'r') as f:
#                 return np.array(f[key])
#         elif filename.endswith('.h5'):
#             with h5py.File(filename, 'r') as f:
#                 return np.array(f[key])
#         else:
#             raise ValueError('No such file format: {}'.format(filename))
    
#     def load_data_list(self):
#         data_list = []
#         for root, dirs, files in os.walk(self.path):
#             for file in files:
#                 path = os.path.join(root, file)
#                 if file.endswith('.hdf5') or file.endswith('.h5'):
#                     with h5py.File(path, 'r') as f:
#                         for key in f.keys():
#                             num_jets_in_file = f[key].shape[0]  # Get number of jets in the file
#                             for jet_idx in range(num_jets_in_file):
#                                 if self.data_files is not None:
#                                     for k in self.data_files.keys():
#                                         if file in self.data_files[k][0] and key == self.data_files[k][1]:
#                                             label = None if self.data_class_labels is None else self.data_class_labels[k]
#                                             data_list.append((k, file, key, jet_idx, label))
#                                 else:
#                                     label = None if self.data_class_labels is None else i
#                                     data_list.append(('sample_{}'.format(i), file, key, jet_idx, label))
#         return data_list

















# class JetNetDataSets(Dataset):

#     ''' Arguments:
#         - `dir_path` : path to data files
#         - `data_files` : dictionary of data files to load, default is `None`
#         - `data_class_labels` : dictionary of class labels for each data file, default is `None`
#         - `num_jets`: number of jets to load, default is `None`
#         - `num_constituents`: number of particle constituents in each jet, default is `150`
#         - `particle_features`: list of particle features to include in data, default is `['eta_rel', 'phi_rel', 'pt_rel']`
#         - `preprocess`: dictionary of preprocessing methods to apply to data, default is `None`
#         - `clip_neg_pt`: clip negative pt values to zero, default is `False`
    
#         Loads and formats data from data files in format `.npy` or `.hdf5`.\\
#         Adds mask and zero-padding if necessary.\\
#         Adds class label for each sample if `data_class_labels` is provided.\\
#         Input data in files should always have shape (`#jets`, `#particles`, `#features`) with feaures: `(eta_rel, phi_rel, pt_rel, ...)`
        
#     '''
    
#     def __init__(self, 
#                  dir_path: str=None, 
#                  data_files: dict=None,
#                  data_class_labels: dict=None,
#                  num_jets: int=None,
#                  num_constituents: int=150,
#                  particle_features: list=['eta_rel', 'phi_rel', 'pt_rel'],
#                  clip_neg_pt: bool=False):
        
#         self.path = dir_path
#         self.data_files = data_files
#         self.data_class_labels = data_class_labels
#         self.num_jets = num_jets
#         self.num_consts = num_constituents
#         self.particle_features = particle_features
#         self.clip_neg_pt = clip_neg_pt
#         self.data_list = self.load_data_list()
        
#     def __len__(self):
#         return len(self.data_list)

#     def __getitem__(self, idx):
#         data, label = self.data_list[idx], self.label_list[idx]
#         if label is None: 
#             return data
#         else:
#             labels = torch.tensor(label, dtype=torch.long)  # Ensure label is a tensor
#             return data, labels

#     def load_file(self, f, key):
#         return np.array(f[key])

#     def load_data_list(self):
#         data_list = []
#         label_list = []
#         for root, dirs, files in os.walk(self.path):
#             for file in files:
#                 path = os.path.join(root, file)
#                 if file.endswith('.hdf5') or file.endswith('.h5'):
#                     with h5py.File(path, 'r') as f:
#                         for key in f.keys():
#                             file_data = self.load_file(path)
#                             for jet_idx in range(file_data.shape[0]):
#                                 data = torch.Tensor(file_data[jet_idx])
#                                 # data = self.preprocess(data)
#                                 data_list.append(data)
#                                 if self.data_files is not None:
#                                     for k in self.data_files.keys():
#                                         if file in self.data_files[k][0] and key == self.data_files[k][1]:
#                                             label = None if self.data_class_labels is None else self.data_class_labels.get(k, None)
#                                             label_list.append(label)
#                                 else:
#                                     label = None if self.data_class_labels is None else i
#                                     label_list.append(label)
#         self.label_list = label_list  # Assign label_list to an instance variable
#         return data_list

#     def preprocess(self, data):

#         N, D = data.shape
            
#         #...fill with zero padding up to 'num_constituents'

#         if D < 3: raise ValueError('Data has wrong shape: {}'.format(data.shape))
#         elif N < self.num_consts: 
#             zero_rows = torch.zeros(self.num_consts - N, D)
#             data = torch.cat((data, zero_rows), dim=0) 

#         #...get particle features

#         pf = {}
#         pf['eta_rel'] = data[:, 0, None]
#         pf['phi_rel'] = data[:, 1, None]
#         pf['pt_rel'] = data[:, 2, None] 
#         pf['log_pt_rel'] = torch.log(pf['pt_rel'])
#         pf['R'] = torch.sqrt(pf['eta_rel']**2 + pf['phi_rel']**2)
#         pf['e_rel'] = pf['pt_rel'] * torch.cosh(pf['eta_rel'])
#         pf['log_e_rel'] = torch.log(pf['e_rel'])
#         features = [pf[f] for f in self.particle_features]

#         #...add mask to indicate which particles are padded

#         mask = (data[:, 0] + data[:, 1] + data[:, 2] != 0).int().unsqueeze(-1)
#         features += [mask]
#         data = torch.cat(features, dim=-1)    
#         data = self._pt_order(data)

#         #...clip negative pt values

#         if self.clip_neg_pt:
#             data_clip = torch.clone(data)    
#             data = torch.zeros_like(data)
#             data[data_clip[:, 2] >= 0.0] = data_clip[data_clip[:, 2] >= 0.0]
#             data = self._pt_order(data)

#         #...preprocess data
#         # TODO implement preprocessing

#         return data[:self.num_consts]

#     def _pt_order(self, data):
#         ''' Order particles in decreasing pt '''
#         return data[torch.argsort(data[..., 2], descending=True)]





# data = JetNetDataSets(dir_path='data/', 
#                         data_files=config.sets,
#                         data_class_labels=config.labels,
#                         num_constituents=30,
#                         clip_neg_pt=True,
#                         particle_features=['eta_rel', 'phi_rel', 'pt_rel'])

# data_loader = DataLoader(dataset=data, batch_size=256, shuffle=True)

# for batch, labels in data_loader: 
#     print(batch, labels)
#     # break

In [ ]:

from src.datasets import JetNetDataSets
from torch.utils.data import DataLoader

class DataConfig:

    sets : dict = {'fm_midpoint' : ('fm_tops150_mp200nfe.h5', 'etaphipt'),
                   'diff_ddim'   : ('ddim_200.h5', 'etaphipt_frac')}

    labels : dict = {'fm_midpoint' : 0,
                     'diff_ddim'   : 1}


config = DataConfig 
data = JetNetDataSets(dir_path='data/', 
                        data_files=config.sets,
                        data_class_labels=config.labels,
                        particle_features=['eta_rel', 'phi_rel', 'pt_rel']
                        )

print(len(data))

data_loader = DataLoader(dataset=data, batch_size=2000, shuffle=True, num_workers=8, pin_memory=True)

for batch, labels in data_loader: 
    print(labels)
    break

In [ ]:
print(data.mean)

# Jet images

In [ ]:
bins = (Bins(-1,1,200), Bins(-1,1,200))

jetnet.image(bins=bins) 
flowmatch.image(bins=bins)
diffusion.image(bins=bins)

# Check distribution of particle features

In [ ]:

data = JetNetDataLoader(dir_path='data/', 
                        data_files=DataConfig.sets,
                        preprocess=None,
                        num_jets=175000,
                        num_constituents=30, 
                        clip_neg_pt=True,
                        particle_features=['eta_rel', 'phi_rel', 'pt_rel', 'R', 'e_rel']
                        )

jetnet = JetNetFeatures(data[2])
flowmatch_mp = JetNetFeatures(data[0])
flowmatch_eu = JetNetFeatures(data[1])
diffusion = JetNetFeatures(data[3])

bins = Bins(0,1)
fig, ax = plt.subplots(1, figsize=(5,5))
jetnet.particle_plot(feature='pt_rel', bins=bins, ax=ax) 
flowmatch_eu.particle_plot(feature='pt_rel', bins=bins,  fill=False, color='r', ax=ax) 
flowmatch_mp.particle_plot(feature='pt_rel', bins=bins,  fill=False, color='purple', ax=ax) 
diffusion.particle_plot(feature='pt_rel', bins=bins, fill=False, color='b', ax=ax) 

bins = Bins(0,3)
fig, ax = plt.subplots(1, figsize=(5,5))
jetnet.particle_plot(feature='R', bins=bins, ax=ax) 
flowmatch_eu.particle_plot(feature='R', bins=bins,  fill=False, color='r', ax=ax) 
flowmatch_mp.particle_plot(feature='R', bins=bins,  fill=False, color='purple', ax=ax) 
diffusion.particle_plot(feature='R', bins=bins, fill=False, color='b', ax=ax) 

bins = Bins(0,0.8)
fig, ax = plt.subplots(1, figsize=(5,5))
jetnet.particle_plot(feature='pt_rel', nth_particle=1, bins=bins, ax=ax) 
flowmatch_eu.particle_plot(feature='pt_rel', nth_particle=1, bins=bins,  fill=False, color='r', ax=ax) 
flowmatch_mp.particle_plot(feature='pt_rel', nth_particle=1,bins=bins,  fill=False, color='purple', ax=ax) 
diffusion.particle_plot(feature='pt_rel', nth_particle=1, bins=bins, fill=False, color='b', ax=ax) 
plt.title(r'hardest particles in jet')

bins = Bins(0,0.35)
fig, ax = plt.subplots(1, figsize=(5,5))
jetnet.jet_plot(feature='m_rel', bins=bins, ax=ax) 
flowmatch_eu.jet_plot(feature='m_rel', bins=bins,  fill=False, color='r', ax=ax) 
flowmatch_mp.jet_plot(feature='m_rel', bins=bins,  fill=False, color='purple', ax=ax) 
diffusion.jet_plot(feature='m_rel', bins=bins, fill=False, color='b', ax=ax) 

bins = range(30)
fig, ax = plt.subplots(1, figsize=(5,5))
jetnet.jet_plot(feature='multiplicity', bins=bins, ax=ax) 
flowmatch_eu.jet_plot(feature='multiplicity', bins=bins,  fill=False, color='r', ax=ax) 
flowmatch_mp.jet_plot(feature='multiplicity', bins=bins,  fill=False, color='purple', ax=ax) 
diffusion.jet_plot(feature='multiplicity', bins=bins, fill=False, color='b', ax=ax) 

# Preprocess data

In [ ]:
data = JetNetDataLoader(dir_path='data/',  data_files=DataConfig.sets, preprocess=DataConfig.preprocess, num_jets=175000, num_constituents=30,  clip_neg_pt=True, particle_features=['eta_rel', 'phi_rel', 'pt_rel', 'R', 'e_rel'])

jetnet = JetNetFeatures(data[2])
flowmatch_mp = JetNetFeatures(data[0])
flowmatch_eu = JetNetFeatures(data[1])
diffusion = JetNetFeatures(data[3])

bins = Bins(-10,50)
fig, ax = plt.subplots(1, figsize=(5,5))
jetnet.particle_plot(feature='pt_rel', bins=bins, ax=ax) 
flowmatch_eu.particle_plot(feature='pt_rel', bins=bins,  fill=False, color='r', ax=ax) 
flowmatch_mp.particle_plot(feature='pt_rel', bins=bins,  fill=False, color='purple', ax=ax) 
diffusion.particle_plot(feature='pt_rel', bins=bins, fill=False, color='b', ax=ax) 

fig, ax = plt.subplots(1, figsize=(5,5))
jetnet.particle_plot(feature='R', bins=bins, ax=ax) 
flowmatch_eu.particle_plot(feature='R', bins=bins,  fill=False, color='r', ax=ax) 
flowmatch_mp.particle_plot(feature='R', bins=bins,  fill=False, color='purple', ax=ax) 
diffusion.particle_plot(feature='R', bins=bins, fill=False, color='b', ax=ax) 

fig, ax = plt.subplots(1, figsize=(5,5))
jetnet.particle_plot(feature='pt_rel', nth_particle=1, bins=bins, ax=ax) 
flowmatch_eu.particle_plot(feature='pt_rel', nth_particle=1, bins=bins,  fill=False, color='r', ax=ax) 
flowmatch_mp.particle_plot(feature='pt_rel', nth_particle=1,bins=bins,  fill=False, color='purple', ax=ax) 
diffusion.particle_plot(feature='pt_rel', nth_particle=1, bins=bins, fill=False, color='b', ax=ax) 
plt.title(r'hardest particles in jet')
